In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from IPython.display import display, clear_output, Markdown
import sys
import transformers
from typing import Optional, List
from threading import Thread
import pandas as pd
import time

/Users/masumbillah/Desktop/virtual_envs/llm/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/masumbillah/Desktop/virtual_envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load model and tokenizer

In [2]:
model_folder = "llama/"
tokenizer_folder = 'processor/'

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_folder)
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set

model = AutoModelForCausalLM.from_pretrained(model_folder, torch_dtype=torch.bfloat16)
model = model.to(torch.device(device))
model.eval()

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.93it/s]


MllamaForCausalLM(
  (model): MllamaTextModel(
    (embed_tokens): Embedding(128264, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-2): 3 x MllamaSelfAttentionDecoderLayer(
        (self_attn): MllamaTextSelfSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MllamaTextMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MllamaTextRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MllamaTextRMSNorm((4096,), eps=1e-05)
     

## Define the class to stream generated tokens

In [3]:
class BatchTextIteratorStreamer(TextIteratorStreamer):
    def __init__(self, batch_size:int, tokenizer: "AutoTokenizer", skip_prompt: bool = False, timeout: Optional[float] = None, **decode_kwargs):
        super().__init__(tokenizer, skip_prompt, timeout, **decode_kwargs)
        self.batch_size = batch_size
        self.token_cache = [[] for _ in range(batch_size)]
        self.print_len = [0 for _ in range(batch_size)]
        self.generate_exception = None

    def put(self, value):
        if len(value.shape) != 2:
            value = torch.reshape(value, (self.batch_size, value.shape[0] // self.batch_size))

        if self.skip_prompt and self.next_tokens_are_prompt:
            self.next_tokens_are_prompt = False
            return

        printable_texts = list()
        for idx in range(self.batch_size):
            self.token_cache[idx].extend(value[idx].tolist())
            text = self.tokenizer.decode(self.token_cache[idx], **self.decode_kwargs)

            if text.endswith("\n"):
                printable_text = text[self.print_len[idx] :]
                self.token_cache[idx] = []
                self.print_len[idx] = 0
                # If the last token is a CJK character, we print the characters.
            elif len(text) > 0 and self._is_chinese_char(ord(text[-1])):
                printable_text = text[self.print_len[idx] :]
                self.print_len[idx] += len(printable_text)
            else:
                printable_text = text[self.print_len[idx] : text.rfind(" ") + 1]
                self.print_len[idx] += len(printable_text)
            printable_texts.append(printable_text)

        self.on_finalized_text(printable_texts)

    def end(self):
        printable_texts = list()
        for idx in range(self.batch_size):
            if len(self.token_cache[idx]) > 0:
                text = self.tokenizer.decode(self.token_cache[idx], **self.decode_kwargs)
                printable_text = text[self.print_len[idx] :]
                self.token_cache[idx] = []
                self.print_len[idx] = 0
            else:
                printable_text = ""
            printable_texts.append(printable_text)

        self.next_tokens_are_prompt = True
        self.on_finalized_text(printable_texts, stream_end=True)

    def on_finalized_text(self, texts: List[str], stream_end: bool = False):
        self.text_queue.put(texts, timeout=self.timeout)
        if stream_end:
            self.text_queue.put(self.stop_signal, timeout=self.timeout)


## Generate outputs

In [43]:
def generate_text(texts, model, tokenizer, device, max_tokens):
    
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
    streamer = BatchTextIteratorStreamer(batch_size = len(texts), tokenizer = tokenizer, skip_special_tokens=True, skip_prompt=True)

    generation_kwargs = dict(inputs, 
                             do_sample=True, 
                             temperature=0.7, 
                             top_k=50, 
                             streamer=streamer,
                             max_new_tokens=max_tokens, 
                             eos_token_id=model.config.eos_token_id)
    
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    for new_text in streamer:
        yield new_text  # Yield the text as it's generated


max_tokens = 100
no_of_users = 50

df = pd.read_csv("dataset/random_questions.csv")
texts = list(df['Question'])
texts = texts[:no_of_users]

start_time = time.time()

answers = ["" for i in range(len(texts))] 
total_time = 0
for new_text in generate_text(texts, model, tokenizer, device, max_tokens):
    # Clear output and update display
    clear_output(wait=True)
    for i in range(len(texts)):
        answers[i] += " " + new_text[i]
        display(Markdown(f"# {texts[i]}\n{answers[i]}"))

total_time = time.time() - start_time
print("Total execution time:", round(total_time,2), "seconds")
print("Avg. token generation time (of the model):", round((total_time/(max_tokens*len(texts)))*1000,2), "ms")
print("Avg. token generation time (per customer):", round((total_time/max_tokens)*1000,2), "ms")

# What are the key concepts in chemistry?
   The  key  concepts  in  chemistry  are  the  fundamental  ideas  that   underlie  the  study  of   chemistry.  These  concepts  are  the  building  blocks  of  chemistry  and  are  essential  for  understanding  the   subject.  Some  of  the  key  concepts  in  chemistry  include:
   1.  Atomic   structure:  The  study  of  the  structure  and  properties  of   atoms,  including  the   nucleus,   electrons,  and  electron  configuration.
   2.  Chemical   bonding:  The  study  of  the  forces  that  hold  atoms  together  to  form   molecules,  including    covalent   bonds,   ionic   bonds,  and  metallic  bonds.
  3

# How has technology evolved over time?
 
  Technology  has  undergone  significant  transformations  since  the  advent  of  the  first   computer.  The  evolution  of  technology  has  been  a  gradual   process,  with  each  innovation  building  upon  the  last  to  create  the  sophisticated  digital  landscape  we  know   today.  Here  are  some  key  milestones  in  the  evolution  of  technology:
   1.      1940s:  The  First  Computer
  The  first  electronic   computer,    ENIAC   (Electronic   Numerical   Integrator  and   Computer),  was  developed  in  the      1940s.  This  massive  machine  weighed  over   27  tons  and  used  vacuum

# How has mathematics evolved over time?
   How  has  it  contributed  to  the  development  of  science  and   technology?  This  course  will  take  you  on  a  journey  through  the  history  of   mathematics,  from  ancient  civilizations  to  modern   times,  exploring  key   concepts,   discoveries,  and   innovations.  You  will  analyze  the  impact  of  mathematical  ideas  on  scientific  and  technological   advancements,  and  examine  the  ways  in  which  mathematics  continues  to  shape  our  understanding  of  the  world  and  our  place  in   it.  This  course  is  designed  to  be   engaging,   interactive,  and  accessible  to  students  with  varying  levels  of  mathematical   background.  No  prior

# Explain the importance of technology in modern society.
   How  has  technology  affected  the  way  people   live,   work,  and  interact  with  one  another?
  The  importance  of  technology  in  modern  society  cannot  be    overstated.  Technology  has   revolutionized  the  way  people   live,   work,  and  interact  with  one   another,  transforming  every  aspect  of  modern   life.  Here  are  some  key  ways  in  which  technology  has  impacted  society:
   1.   Communication:  Technology  has  made  it  possible  for  people  to  communicate  with  each  other   instantly,  regardless  of  their  geographical   location.  With  the  advent  of  social   media,  messaging   apps,  and

# How can philosophy be applied in real-world scenarios?
   Philosophy  has  been  a  part  of  human  inquiry  for   centuries,  and  its  influence  can  be  seen  in  many  areas  of   life.  Here  are  some  examples  of  how  philosophy  can  be  applied  in   real-world  scenarios:

   1.       **Decision-making**:  Philosophy  can  help  individuals  make  informed  decisions  by  considering  various   perspectives,  evaluating   evidence,  and  weighing  the  pros  and  cons  of  different  options.
   2.       **Ethics**:   Philosophical  principles  can  guide  individuals  in  making  ethical   decisions,  such  as  resolving  moral    dilemmas,  applying  moral  principles  to  complex

...

# What are some ethical concerns related to psychology?
 
  There  are  several  ethical  concerns  related  to   psychology.  Here  are  some  of  the  most  significant  ones:
   1.   Informed   Consent:   Ensuring  that  clients  or  research  participants  understand  the   purpose,   risks,  and  benefits  of  a  study  or   treatment,  and  provide  their  consent  before  participating.
   2.    Confidentiality:   Protecting  the  privacy  of  clients  or  research   participants,  and  maintaining  confidentiality  of  their  personal  information.
   3.  Respect  for    Autonomy:   Respecting  the  rights  and  decisions  of  clients  or  research   participants,  and  avoiding  coercion  or  manipulation

# How has geography evolved over time?
 
    Geography,  as  a  field  of   study,  has  undergone  significant  changes  and  developments  over   time.  From  its  origins  in  ancient  civilizations  to  its  current  status  as  a    multidisciplinary   field,  geography  has  evolved  in  response  to  changing  societal   needs,  technological   advancements,  and  shifts  in  the  way  we  understand  the   world.   Here's  a  brief  overview  of  the  major  developments  in  the  evolution  of  geography:

    **Ancient  Period    (3000  BCE  -   500   CE)**

  *  Geography  originated  in  ancient   civilizations,  such  as    Mesopotam

# What are some ethical concerns related to chemistry?
  What  are  some  ethical  concerns  related  to  chemistry?
   Chemistry  has  a  wide  range  of  applications  and  can  be  used  to  develop  new   technologies,  improve  our  understanding  of  the  natural   world,  and  provide  solutions  to  various  environmental  and  health   problems.   However,  with  these  benefits  come  ethical  concerns  that  need  to  be   addressed.  Here  are  some  of  the  most  significant  ethical  concerns  related  to  chemistry:
   1.     **Toxicity  and  Environmental   Impact**:  Many  chemicals  used  in  industrial   processes,  agricultural   applications,  and  consumer  products  can  be  harmful  to  human  health

# What are some ethical concerns related to ai?
 
  There  are  several  ethical  concerns  related  to   AI,  including:
   1.  Bias  and   discrimination:  AI  systems  can   perpetuate  and  even  amplify  existing  biases  and  discrimination  if  they  are  trained  on  biased  data  or  designed  with  a  particular  worldview.
   2.  Job   displacement:  AI  has  the  potential  to  automate  many   jobs,  which  could  lead  to  significant  job  displacement  and  social  disruption.
   3.  Lack  of  transparency  and   accountability:  AI  systems  can  be  complex  and  difficult  to   understand,  making  it  challenging  to  determine  how  they  make  decisions  and  who  is  responsible

Total execution time: 27.49 seconds
Avg. token generation time (of the model): 5.5 ms
Avg. token generation time (per customer): 274.9 ms
